In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained deepfake detection models
model_CNN = load_model('deepfake_detection_complete_CNN_model.h5')
model_ResNet = load_model('deepfake_detection_ResNet50V2_final_model.h5')
model_VAE = load_model('deepfake_detection_complete_OC-VAE_model.h5')

threshold_vae = 0.2636  # Set the threshold for VAE model

def preprocess_image(img, target_size):
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def classify_image():
    file_path = filedialog.askopenfilename()
    
    if file_path:
        try:
            img = Image.open(file_path)
            
            # Preprocess the image for CNN model
            img_cnn = preprocess_image(img, (256, 256))
            prediction_cnn = model_CNN.predict(img_cnn)
            print(prediction_cnn)

            # Preprocess the image for ResNet model
            img_resnet = preprocess_image(img, (100, 100))
            prediction_resnet = model_ResNet.predict(img_resnet)
            print(prediction_resnet)

            # Evaluate the image using the VAE model
            img_vae = preprocess_image(img, (100, 100))  # Adjust target size as required by VAE model
            loss_vae = model_VAE.evaluate(img_vae)
            
            # Hard voting
            prediction_cnn_label = 1 if prediction_cnn > 0.5 else 0
            prediction_resnet_label = 1 if prediction_resnet > 0.5 else 0
            prediction_vae_label = 0 if loss_vae < threshold_vae else 1

            # Count the number of models predicting 'Real'
            num_models_real = prediction_cnn_label + prediction_resnet_label + prediction_vae_label

            # If at least 2 models predict 'Real', classify as 'Real', otherwise classify as 'Fake'
            prediction_label = 'Fake' if num_models_real >= 2 else 'Real'           

            # Display the result
            result_label.config(text=f"Prediction: {prediction_label}", fg="green" if prediction_label == 'Real' else "red")
            
            # Display the image
            img = img.resize((300, 300))
            img = ImageTk.PhotoImage(img)
            img_label.config(image=img)
            img_label.image = img
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

root = tk.Tk()
root.title("Deepfake Detection")
root.geometry("600x500")
root.configure(bg="#1E1E1E")

title_label = tk.Label(root, text="Deepfake Detection", font=("Helvetica", 24, "bold"), fg="#FFD700", bg="#1E1E1E")
title_label.pack(pady=20)

select_button = tk.Button(root, text="Select Image", command=classify_image, bg="#4CAF50", fg="white", font=("Helvetica", 14, "bold"), relief=tk.FLAT)
select_button.pack(pady=10)

result_label = tk.Label(root, text="", font=("Helvetica", 16), bg="#1E1E1E", fg="#FFD700")
result_label.pack()

img_label = tk.Label(root, bg="#1E1E1E")
img_label.pack()

root.mainloop()






1/1 [==============================] - 0s 229ms/step
[[0.99994975]]
1/1 [==============================] - 2s 2s/step
[[0.9574706]]
1/1 [==============================] - 1s 650ms/step - loss: 0.2463
